In [1]:
## print('Starting generic python imports')
from pathlib import Path
import numpy as np
import os
from time import time, sleep
import sys
from munch import Munch
from itertools import product, islice

## overall input parameters
repeats = 5
Ts = [100, 10, 2, 1.75, 1.5, 1.25]
Ns = np.logspace(np.log10(35), np.log10(250), 10, dtype = np.int) // 5 * 5

initial_states_cycle = [
    lambda N: np.arange(N, dtype = np.float64) % 2, #a CDW initial state
    #lambda N: np.zeros(N, dtype = np.float64),
    #lambda N: np.random.choice(np.array([0,1], dtype = np.float64), size = N),
]

structure_names = ('repeats', 'states', 'Ts', 'Ns')
lengths = (repeats, len(initial_states_cycle), len(Ts))
prod = product(*[range(l) for l in lengths])

J = 5
U = 10
alpha = 1.25


total_jobs = len(Ts) * repeats * len(initial_states_cycle)

#give information to the dispatch script
batch_params = Munch(total_jobs = total_jobs, repeats = repeats, Ts = Ts, structure_names = structure_names)

print('Doing FKMC imports')
from FKMC.montecarlo import *

print('Getting environment variables')
slurm_job_id = int(os.getenv('SLURM_ARRAY_JOB_ID', -1))
slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_ID', 11))
max_slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_MAX', total_jobs - 1))
working_dir = Path(".")

##sleep if necessary3-400
debug = (slurm_job_id == -1)
sleeptime = np.random.random() * (30)
print(f'Waiting for {sleeptime:.0f} seconds to randomise the finish time')
if not debug: sleep(sleeptime)


##using the value of slurm_task_id, decide what parameters this job will have
repeat_i, state_i, T_i = next(islice(prod, slurm_task_id, None))
make_initial_state = initial_states_cycle[state_i]
T = Ts[T_i]


print(f'T_i = {T_i}, repeat_i = {repeat_i}, state_i = {state_i}')
print(f'total_jobs = {total_jobs} max_slurm_task_id = {max_slurm_task_id}')

logs = np.empty(shape = len(Ns), dtype = object)

t1 = time()
for i, N in enumerate(Ns):
    parameters = dict(t = 1, alpha = alpha, mu = 0, beta = 1/T, J = J, U = U, normalise = True)
    MCMC_params = dict(
            state = make_initial_state(N),
            N_steps = int(1e4) if not debug else 100,
            N_burn_in = int(1e3) if not debug else 0,
            thin = 10 if not debug else 1,
            logger = Eigenspectrum_IPR_all(bins = 2000, limit = 20),
            proposal = p_multi_site_uniform_reflect,
            accept_function = perturbation_accept,
            warnings = True,
        )

    print(f'starting N = {N}')
    t0 = time()
    logs[i] = FK_mcmc(**MCMC_params, parameters = parameters)
    logs[i].time = time() - t0

    parameters['J_matrix'] = '...'
    MCMC_params['state'] = '...'
    print(f'''
    params = {parameters},
    mc_params = {MCMC_params}
    ''')

    filename = f'{slurm_task_id}.npz'

    print(f'This N = {N} took {time() - t0:.0f} seconds.')

    if not debug: 
        print(f'Saving in {filename}')
        np.savez_compressed(working_dir/filename, 
        Ns = Ns, Ts = Ts, parameters = parameters, MCMC_params = MCMC_params, logs = logs, allow_pickle = True,
        desc = ''
        )

print(f'Overall: {time() - t1:.0f} seconds.')

Doing FKMC imports
Getting environment variables
Waiting for 3 seconds to randomise the finish time
T_i = 5, repeat_i = 1, state_i = 0
total_jobs = 30 max_slurm_task_id = 29
starting N = 35
0% through after 0.00m
10% through after 0.00m
20% through after 0.01m
30% through after 0.01m
40% through after 0.01m
50% through after 0.01m
60% through after 0.02m
70% through after 0.02m
80% through after 0.02m
90% through after 0.02m
Warning, p_acc = 0.12885714285714286, {'t': 1, 'alpha': 1.25, 'mu': 0, 'beta': 0.8, 'J': 5, 'U': 10, 'normalise': True, 'J_matrix': 'suppressed for brevity'}

    params = {'t': 1, 'alpha': 1.25, 'mu': 0, 'beta': 0.8, 'J': 5, 'U': 10, 'normalise': True, 'J_matrix': '...'},
    mc_params = {'state': '...', 'N_steps': 100, 'N_burn_in': 0, 'thin': 1, 'logger': <FKMC.montecarlo.Eigenspectrum_IPR_all object at 0x2ab7edb42748>, 'proposal': <function p_multi_site_uniform_reflect at 0x2ab7edb511e0>, 'accept_function': <function perturbation_accept at 0x2ab7edb51510>, 'warn